# Community
## "Where do I belong?"
- Definition of Community and Community Detection
- Examples
- Perform community detection on dataset
    - k-cliques or Girvan-Newman


In [6]:
import networkx as nx
from networkx.algorithms.community import k_clique_communities

In [3]:
path = '../data/'

In [12]:
GA = nx.read_gexf(path + 'ga_graph.gexf')

In [23]:
list(k_clique_communities(GA, 2))

[frozenset({'bailey', 'ben', 'tucker'}),
 frozenset({'addison',
            'altman',
            'arizona',
            'avery',
            'colin',
            'denny',
            'derek',
            'finn',
            'grey',
            'hank',
            'izzie',
            'karev',
            'kepner',
            'lexi',
            'mrs. seabury',
            'nancy',
            "o'malley",
            'olivia',
            'owen',
            'preston',
            'sloan',
            'steve',
            'torres',
            'yang'}),
 frozenset({'adele', 'chief', 'ellis grey', 'susan grey', 'thatch grey'})]